In [63]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

mnist = input_data.read_data_sets('../data/MNIST_data', one_hot=True)

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [64]:
tf.reset_default_graph()

In [65]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def sample_z(n,m):
    return np.random.uniform(-1.,1., size=[n,m])

y_dim = mnist.train.labels.shape[1]
y = tf.placeholder(tf.float32, shape=[None, y_dim])

In [66]:
# Defining input placeholder and parameters for the generator

Z = tf.placeholder(tf.float32, shape=[None, 100])
with tf.name_scope("generator"):
    G_W1 = tf.Variable(xavier_init([100+y_dim, 128]),name="G_W1")
    G_b1 = tf.Variable(tf.zeros(shape=[128]),name="G_B1")

    G_W2 = tf.Variable(xavier_init([128, 784]),name="G_W2")
    G_b2 = tf.Variable(tf.zeros(shape=[784]),name="G_B2")

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [67]:
# Defining input placeholder and parameters for the discriminator

X = tf.placeholder(tf.float32, shape=[None, 784])

with tf.name_scope("discriminator"):

    D_W1 = tf.Variable(xavier_init([784+y_dim, 128]), name="D_W1")
    D_b1 = tf.Variable(tf.zeros(shape=[128]),name="D_B1")

    D_W2 = tf.Variable(xavier_init([128, 1]),name="D_W2")
    D_b2 = tf.Variable(tf.zeros(shape=[1]),name="D_B2")

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [68]:
def generator(z,y):
    with tf.name_scope("generator"):
        x = tf.concat([z,y],axis=1)
        G_h1 = tf.nn.relu(tf.matmul(x, G_W1) + G_b1)
        G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
        G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [69]:
def discriminator(x,y):
    with tf.name_scope("discriminator"):
        x = tf.concat([x,y],axis=1)
        D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
        D_log_prob = tf.matmul(D_h1, D_W2) + D_b2
        D_prob = tf.nn.sigmoid(D_log_prob)
    return D_prob

In [70]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [71]:
G_sample = generator(Z,y)
D_real = discriminator(X,y)
D_fake = discriminator(G_sample,y)

In [72]:
D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))

In [73]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list = theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list = theta_G)

In [74]:
z_size = 100
mb_size = 16

In [75]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [76]:
if not os.path.exists('cgan_out'):
    os.makedirs('cgan_out')

i = 0

for it in range(1000000):
    if it % 100 == 0:
        current_y = np.zeros(shape=[16,y_dim])
        current_y[:,7] = 1
        samples = sess.run(G_sample, feed_dict={Z: sample_z(16, z_size),y:current_y})

        fig = plot(samples)
        plt.savefig('cgan_out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, y_mb = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_z(mb_size, z_size), y: y_mb})
    
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_z(mb_size, z_size), y:y_mb})

    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.265
G_loss: 2.588

Iter: 100
D loss: 0.1774
G_loss: 3.483

Iter: 200
D loss: 0.1061
G_loss: 4.286

Iter: 300
D loss: 0.05358
G_loss: 5.875

Iter: 400
D loss: 0.07068
G_loss: 4.451

Iter: 500
D loss: 0.01712
G_loss: 4.456

Iter: 600
D loss: 0.01973
G_loss: 5.703

Iter: 700
D loss: 0.07594
G_loss: 6.877

Iter: 800
D loss: 0.00611
G_loss: 7.35

Iter: 900
D loss: 0.002889
G_loss: 8.567

Iter: 1000
D loss: 0.000978
G_loss: 9.45

Iter: 1100
D loss: 0.005879
G_loss: 7.391

Iter: 1200
D loss: 0.003922
G_loss: 11.66

Iter: 1300
D loss: 0.002529
G_loss: 7.328

Iter: 1400
D loss: 0.004361
G_loss: 7.99

Iter: 1500
D loss: 0.005762
G_loss: 9.697

Iter: 1600
D loss: 0.01195
G_loss: 6.25

Iter: 1700
D loss: 0.005909
G_loss: 8.363

Iter: 1800
D loss: 0.007851
G_loss: 8.253

Iter: 1900
D loss: 0.003122
G_loss: 10.2

Iter: 2000
D loss: 0.07981
G_loss: 6.912

Iter: 2100
D loss: 0.03087
G_loss: 7.297

Iter: 2200
D loss: 0.009698
G_loss: 8.442

Iter: 2300
D loss: 0.03394
G_loss: 5.491

It

KeyboardInterrupt: 